#**Ввод данных**

In [1]:
!unzip data_input4.zip
!unzip data_input5.zip

Archive:  data_input4.zip
   creating: data_input4/
   creating: data_input4/01_healthy/
  inflating: data_input4/01_healthy/cortex_patient13012020_633nm_obj20_power100_1sec_acc40_сenter2900_place1_20201009_1.txt  
  inflating: data_input4/01_healthy/cortex_patient13012020_633nm_obj20_power100_1sec_сenter2900_place1_20201009_1.txt  
  inflating: data_input4/01_healthy/cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place1_20210202.txt  
  inflating: data_input4/01_healthy/cortex_patient15012020_633nm_obj20_power100_1sec_acc1_сenter2900_place2_20210202.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place10_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place1_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_patient180219_633nm_obj50_power100_1sec_acc40_сenter2900_place2_20200813_1.txt  
  inflating: data_input4/01_healthy/cortex_pat

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import graphviz, pydot, pydotplus
from sklearn.model_selection import train_test_split
import joblib

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [4]:
root_folder = "data_input4/01_healthy"
healthy = []
for root, dirs, files in os.walk(root_folder):  
    for filename in files:
        healthy.append(filename)

In [5]:
measurements = []
classifications = []
count_healhy = 0
for sample in healthy:
    df = pd.read_csv(os.path.join(root_folder,sample), sep='\t',skiprows=[0],
        header=None, names=[ 'Wave', 'Intensity'])
    measurements.append(df[['Intensity']].to_numpy())
    classifications.append([1,0])
    count_healhy += 1

In [6]:
sick_path = 'image_tumor_patient20022019_633nm_obj20_power100_1sec_сenter2900.txt'
count_sick = 0
df = pd.read_csv(sick_path, sep='\t',skiprows=[0],
    header=None, names=['X', 'Y', 'Wave', 'Intensity'])
for i in range(456):
    measurements.append(df[['Intensity']][i*len(df['Wave'].unique()):(i+1)*len(df['Wave'].unique())].to_numpy())
    classifications.append([0,1])
    count_sick += 1



In [7]:
healthy_path_1 = 'data_input5/healthy/1.txt'
healthy_path_2 = 'data_input5/healthy/2.txt'

sick_path_1 = 'data_input5/sick/1.txt'
sick_path_2 = 'data_input5/sick/2.txt'

df = pd.read_csv(healthy_path_1, sep='\t',skiprows=[0],
    header=None, names=['X', 'Y', 'Wave', 'Intensity'])

In [8]:
len(df['Wave'].unique())

1015

In [9]:
df.shape[0] / 1015

468.0

In [10]:
for i in range(468):
    measurements.append(df[['Intensity']][i*len(df['Wave'].unique()):(i+1)*len(df['Wave'].unique())].to_numpy())
    classifications.append([1,0])
    count_healhy += 1

In [11]:
df = pd.read_csv(healthy_path_2, sep='\t',skiprows=[0],
    header=None, names=['X', 'Y', 'Wave', 'Intensity'])

In [12]:
df.shape[0] / 1015

468.0

In [13]:
for i in range(468):
    measurements.append(df[['Intensity']][i*len(df['Wave'].unique()):(i+1)*len(df['Wave'].unique())].to_numpy())
    classifications.append([1,0])
    count_healhy += 1

In [14]:
df = pd.read_csv(sick_path_1, sep='\t',skiprows=[0],
    header=None, names=['X', 'Y', 'Wave', 'Intensity'])

In [15]:
df.shape[0] / 1015

468.0

In [16]:
for i in range(468):
    measurements.append(df[['Intensity']][i*len(df['Wave'].unique()):(i+1)*len(df['Wave'].unique())].to_numpy())
    classifications.append([0,1])
    count_sick += 1

In [17]:
df = pd.read_csv(sick_path_2, sep='\t',skiprows=[0],
    header=None, names=['X', 'Y', 'Wave', 'Intensity'])
df.shape[0] / 1015

468.0

In [18]:
for i in range(468):
    measurements.append(df[['Intensity']][i*len(df['Wave'].unique()):(i+1)*len(df['Wave'].unique())].to_numpy())
    classifications.append([0,1])
    count_sick += 1

In [19]:
print(f'Больные ткани: {count_sick}')
print(f'Здоровые ткани: {count_healhy}')

Больные ткани: 1392
Здоровые ткани: 1368


In [20]:
X = np.asarray(measurements)
y = np.asarray(classifications)

In [21]:
X.shape

(2760, 1015, 1)

In [22]:
X = X.reshape(2760, 1015)
X.shape

(2760, 1015)

In [23]:
y.shape

(2760, 2)

In [24]:
#check for duplicates
len(np.unique(X)) < len(X)

False

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(f'Total amount of train measurements: {X_train.shape}')
print(f'Total amount of train labels: {y_train.shape}')
print(f'Total amount of test measurements: {X_test.shape}')
print(f'Total amount of test labels: {y_test.shape}')
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

Total amount of train measurements: (2070, 1015)
Total amount of train labels: (2070, 2)
Total amount of test measurements: (690, 1015)
Total amount of test labels: (690, 2)


In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import LocallyLinearEmbedding
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator

# **Блендинговые архитектуры**:

In [78]:
class BoostAndLogisticStack(BaseEstimator):
  def __init__(self, logistic_model_params: dict = None, boosting_model_params: dict = None, verbose: bool = False):
    self.boosting_model = CatBoostClassifier(task_type="GPU", verbose=verbose)
    #self.boosting_model = lgb.LGBMClassifier()
    self.boosting_model_params = boosting_model_params
    #self.boosting_model = xgb.XGBClassifier()
    self.logistic_model = LogisticRegression(max_iter=10000)
    self.logistic_model_params = logistic_model_params
    self.verbose = verbose

  def fit(self, X, y):
    self.boosting_model.fit(X, y)
    self.logistic_model.fit(X, y)
    pass

  def predict(self, X):
    naive_output = (self.boosting_model.predict_proba(X) + self.logistic_model.predict_proba(X)) / 2
    return np.argmax(naive_output, axis=1)


class BoostWrapper(BaseEstimator):
  def __init__(self, model_params: dict = None, verbose: bool = False):
    #self.boosting_model = CatBoostClassifier(task_type="GPU", verbose=False)
    if model_params is None:
      self.boosting_model = lgb.LGBMClassifier()
    else:
      self.boosting_model = lgb.LGBMClassifier(**boosting_model_params)
    self.model_params = model_params
    #self.boosting_model = xgb.XGBClassifier()
    self.verbose = verbose

  def fit(self, X, y):
    self.boosting_model.fit(X, y)
    pass

  def predict(self, X):
    return self.boosting_model.predict(X)


class BoostAndKnnStack(BaseEstimator):
  def __init__(self, boosting_model_params: dict = None, knn_model_params: dict = None, verbose: bool = False):
    self.boosting_model_params = boosting_model_params
    self.knn_model_params = knn_model_params

    if self.boosting_model_params == None:
      self.boosting_model = lgb.LGBMClassifier()
    else:
      self.boosting_model = lgb.LGBMClassifier(**self.boosting_model_params)

    if self.knn_model_params == None:
      self.knn_model = KNeighborsClassifier()
    else:
      self.knn_model = KNeighborsClassifier(**self.knn_model_params)
    self.verbose = verbose

  def fit(self, X, y):
    self.boosting_model.fit(X, y)
    self.knn_model.fit(X, y)
    pass

  def predict(self, X):
    naive_output = (self.boosting_model.predict_proba(X) + self.knn_model.predict_proba(X)) / 2
    return np.argmax(naive_output, axis=1)

  def predict_proba(self, X):
    return (self.boosting_model.predict_proba(X) + self.knn_model.predict_proba(X)) / 2


class BoostAndLogisticAndKnnStack(BaseEstimator):
  def __init__(self, logistic_model_params: dict = None, boosting_model_params: dict = None, knn_model_params: dict = None, verbose: bool = False):
    self.boosting_model_params = boosting_model_params
    self.logistic_model_params = logistic_model_params
    self.knn_model_params = knn_model_params

    if self.boosting_model_params == None:
      self.boosting_model = lgb.LGBMClassifier()
    else:
      self.boosting_model = lgb.LGBMClassifier(**self.boosting_model_params)

    if self.logistic_model_params == None:
      self.logistic_model = LogisticRegression(max_iter=10000000)
    else:
      self.logistic_model = LogisticRegression(**self.logistic_model_params)

    if self.knn_model_params == None:
      self.knn_model = KNeighborsClassifier()
    else:
      self.knn_model = KNeighborsClassifier(**self.knn_model_params)
    self.verbose = verbose

  def fit(self, X, y):
    self.boosting_model.fit(X, y)
    self.logistic_model.fit(X, y)
    self.knn_model.fit(X, y)
    pass

  def predict(self, X):
    naive_output = (self.boosting_model.predict_proba(X) + self.logistic_model.predict_proba(X) + self.knn_model.predict_proba(X)) / 3
    return np.argmax(naive_output, axis=1)


class BoostAndLogisticAndKnnStackNeural(BaseEstimator):
  def __init__(self, logistic_model_params: dict = None, boosting_model_params: dict = None, knn_model_params: dict = None, verbose: bool = False, nn_model = None, n_epoch = 10):
    self.boosting_model_params = boosting_model_params
    self.logistic_model_params = logistic_model_params
    self.knn_model_params = knn_model_params

    if self.boosting_model_params == None:
      self.boosting_model = lgb.LGBMClassifier()
    else:
      self.boosting_model = lgb.LGBMClassifier(**self.boosting_model_params)

    if self.logistic_model_params == None:
      self.logistic_model = LogisticRegression(max_iter=100000)
    else:
      self.logistic_model = LogisticRegression(**self.logistic_model_params)

    if self.knn_model_params == None:
      self.knn_model = KNeighborsClassifier()
    else:
      self.knn_model = KNeighborsClassifier(**self.knn_model_params)

    self.verbose = verbose
    self.verbose = verbose
    self.nn_model = nn_model
    self.n_epoch = n_epoch

  def fit(self, X, y):
    self.boosting_model.fit(X, y)
    self.logistic_model.fit(X, y)
    self.knn_model.fit(X, y)

    boosting_proba = self.boosting_model.predict_proba(X)
    logistic_proba = self.logistic_model.predict_proba(X)
    knn_proba = self.knn_model.predict_proba(X)
    train_metafeatures = np.concatenate((boosting_proba, logistic_proba, knn_proba), axis=1)
    #gotta format y to y_neural
    #healthy == [1, 0]
    #sick == [0, 1]
    y_neural = np.empty([y.size, 2])
    i = 0
    for e in y:
      if e == 0:
          y_neural[i] = np.array([1, 0])
      if e == 1:
          y_neural[i] = np.array([0, 1])
      i += 1
    y_neural = y_neural.astype(int)

    self.nn_model.fit(train_metafeatures, y_neural, epochs=self.n_epoch, batch_size=16)
    pass

  def predict(self, X):
    boosting_proba = self.boosting_model.predict_proba(X)
    logistic_proba = self.logistic_model.predict_proba(X)
    knn_proba = self.knn_model.predict_proba(X)
    test_metafeatures = np.concatenate((boosting_proba, logistic_proba, knn_proba), axis=1)
    nn_output = self.nn_model.predict(test_metafeatures)
    prediction = np.argmax(nn_output, axis=1)
    return prediction


In [50]:
knn_params = {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
skf = StratifiedKFold(n_splits=5, shuffle=True)
y_labels = np.argmax(y, axis=1)
steps = [('scaler', StandardScaler()), ('svd', TruncatedSVD(n_components=10)), ('m', BoostAndKnnStack(knn_model_params=knn_params))]
model = Pipeline(steps=steps).fit(X, y_labels)

In [ ]:
from joblib import load, dump
dump(model, 'final_final_model.joblib') 

['final_model.joblib']

In [ ]:
loaded = load('model.joblib')

In [ ]:
preds = loaded.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(preds, y_labels)

1.0

In [59]:
knn_params = {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
skf = StratifiedKFold(n_splits=5, shuffle=True)
y_labels = np.argmax(y, axis=1)
steps = [('scaler', StandardScaler()), ('svd', TruncatedSVD(n_components=10)), ('m', BoostAndKnnStack(knn_model_params=knn_params))]
model = Pipeline(steps=steps)
results = cross_validate(model, X, y_labels, cv=skf, scoring='accuracy', return_train_score=True)

In [60]:
np.mean(results['test_score'])

0.9561594202898551

#**Подбор гиперпараметров**

In [ ]:
from sklearn.model_selection import GridSearchCV 
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
# define grid search
#grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)}
grid = { 'm__n_neighbors' : n_neighbors, 'm__weights' : weights, 'm__metric' : metric}
cv = StratifiedKFold(n_splits=3, random_state=1, shuffle=True)
steps = [('scaler', StandardScaler()), ('svd', TruncatedSVD(n_components=10)), ('m', KNeighborsClassifier())]
model = Pipeline(steps=steps)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y_labels)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.873551 using {'m__metric': 'manhattan', 'm__n_neighbors': 9, 'm__weights': 'distance'}
0.816304 (0.009600) with: {'m__metric': 'euclidean', 'm__n_neighbors': 1, 'm__weights': 'uniform'}
0.815942 (0.009490) with: {'m__metric': 'euclidean', 'm__n_neighbors': 1, 'm__weights': 'distance'}
0.807246 (0.018708) with: {'m__metric': 'euclidean', 'm__n_neighbors': 3, 'm__weights': 'uniform'}
0.817029 (0.015122) with: {'m__metric': 'euclidean', 'm__n_neighbors': 3, 'm__weights': 'distance'}
0.806884 (0.007754) with: {'m__metric': 'euclidean', 'm__n_neighbors': 5, 'm__weights': 'uniform'}
0.818116 (0.012115) with: {'m__metric': 'euclidean', 'm__n_neighbors': 5, 'm__weights': 'distance'}
0.814130 (0.013253) with: {'m__metric': 'euclidean', 'm__n_neighbors': 7, 'm__weights': 'uniform'}
0.827536 (0.013174) with: {'m__metric': 'euclidean', 'm__n_neighbors': 7, 'm__weights': 'distance'}
0.812681 (0.012049) with: {'m__metric': 'euclidean', 'm__n_neighbors': 9, 'm__weights': 'uniform'}
0.829710 (

In [ ]:
!pip install optuna   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.6 MB/s eta 0:00:00


In [ ]:
from optuna.integration import LightGBMPruningCallback
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import optuna

y_labels = np.argmax(y, axis=1)

def objective(trial, X, y):
    boosting_param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    knn_params = {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
    skf = StratifiedKFold(n_splits=3, shuffle=True)
    y_labels = np.argmax(y, axis=1)
    steps = [('scaler', StandardScaler()), ('svd', TruncatedSVD(n_components=10)), ('m', BoostAndKnnStack(boosting_model_params=boosting_param_grid, knn_model_params=knn_params))]
    model = Pipeline(steps=steps)
    results = cross_validate(model, X, y_labels, cv=skf, scoring='accuracy', return_train_score=True)


    return np.mean(results['test_score'])

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=200)

[I 2023-04-17 21:19:30,899] A new study created in memory with name: LGBM Classifier
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[LightGBM] [Warning] min_gain_to_split is set=7.187355186437564, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.187355186437564
[LightGBM] [Warning] lambda_l2 is set=80, reg_lambda=0.0 will be ignored. Current value: lambda_l2=80
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=6200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction i

[I 2023-04-17 21:19:34,885] Trial 0 finished with value: 0.868840579710145 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2910683119762668, 'num_leaves': 1740, 'max_depth': 3, 'min_data_in_leaf': 6200, 'lambda_l1': 45, 'lambda_l2': 80, 'min_gain_to_split': 7.187355186437564, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 0 with value: 0.868840579710145.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=50, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=12.305540825497237, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=12.305540825497237
[LightGBM] [Warning] lambda_l2 is set=95, reg_lambda=0.0 will be ignored. Current value: lambda_l2=95
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7800
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=50, reg_alpha=0.0 will be ignored. Current value: lambda_l1=50
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 wil

[I 2023-04-17 21:19:37,671] Trial 1 finished with value: 0.8637681159420291 and parameters: {'n_estimators': 10000, 'learning_rate': 0.13239815592393514, 'num_leaves': 2020, 'max_depth': 5, 'min_data_in_leaf': 7800, 'lambda_l1': 50, 'lambda_l2': 95, 'min_gain_to_split': 12.305540825497237, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 0.868840579710145.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_gain_to_split is set=4.879731244551776, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.879731244551776
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=2600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2600
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=70, reg_alpha=0.0 will be ignored. Current value: lambda_l1=70
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will 

[I 2023-04-17 21:27:44,092] Trial 157 finished with value: 0.8666666666666667 and parameters: {'n_estimators': 10000, 'learning_rate': 0.2971174485315636, 'num_leaves': 2360, 'max_depth': 3, 'min_data_in_leaf': 2600, 'lambda_l1': 70, 'lambda_l2': 10, 'min_gain_to_split': 4.879731244551776, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.9}. Best is trial 67 with value: 0.8778985507246378.
/usr/local/lib/python3.9/dist-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=60, reg_alpha=0.0 will be ignored. Current value: lambda_l1=60
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] min_gain_to_split is set=5.728594389830755, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.728594389830755
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=3100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=60, reg_alpha=0.0 will be ignored. Current value: lambda_l1=60
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will 

[W 2023-04-17 21:27:46,593] Trial 158 failed with parameters: {'n_estimators': 10000, 'learning_rate': 0.27891059749616787, 'num_leaves': 2540, 'max_depth': 3, 'min_data_in_leaf': 3100, 'lambda_l1': 60, 'lambda_l2': 10, 'min_gain_to_split': 5.728594389830755, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.9} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-174-8be10d6dc4d2>", line 2, in <lambda>
    func = lambda trial: objective(trial, X, y)
  File "<ipython-input-173-753fd117ede3>", line 33, in objective
    results = cross_validate(model, X, y_labels, cv=skf, scoring='accuracy', return_train_score=True)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 266, in cross_validate
    results = parallel(
  File "/usr/local/lib/python3.9

KeyboardInterrupt: ignored

In [ ]:
results

{'fit_time': array([8.27876091, 8.37650156, 8.07955146, 8.11989784, 7.77592921]),
 'score_time': array([0.02586532, 0.02231216, 0.02947664, 0.03370523, 0.02291965]),
 'test_score': array([0.9494382 , 0.92134831, 0.97191011, 0.94915254, 0.89830508]),
 'train_score': array([0.99859155, 0.9971831 , 1.        , 0.99718706, 0.99718706])}

# **Эксперименты с нейронным слоем**:

In [ ]:
#model 1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dropout
skf = StratifiedKFold(n_splits=4, shuffle=True)
y_labels = np.argmax(y, axis=1)

#model 1
nn_model = Sequential()
nn_model.add(Dense(8, input_shape=(6,), activation='relu'))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(2, activation='sigmoid'))
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

boosting_params = {'n_estimators': 10000,
 'learning_rate': 0.15138872524701985,
 'num_leaves': 1640,
 'max_depth': 8,
 'min_data_in_leaf': 200,
 'lambda_l1': 45,
 'lambda_l2': 70,
 'min_gain_to_split': 5.641360966853382,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.5}


logistic_params = {'max_iter' : 100000, 'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}

knn_params = {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}

model = BoostAndLogisticAndKnnStackNeural(nn_model=nn_model, n_epoch=45, boosting_model_params=boosting_params, logistic_model_params=logistic_params, knn_model_params=knn_params)
results = cross_validate(model, X, y_labels, cv=skf, scoring='accuracy', return_train_score=True)

/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

Epoch 1/45
42/42 [==============================] - 3s 3ms/step - loss: 0.5764 - accuracy: 1.0000
Epoch 2/45
42/42 [==============================] - 0s 3ms/step - loss: 0.4509 - accuracy: 1.0000
Epoch 3/45
42/42 [==============================] - 0s 3ms/step - loss: 0.3172 - accuracy: 1.0000
Epoch 4/45
42/42 [==============================] - 0s 3ms/step - loss: 0.2035 - accuracy: 1.0000
Epoch 5/45
42/42 [==============================] - 0s 3ms/step - loss: 0.1244 - accuracy: 1.0000
Epoch 6/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0762 - accuracy: 1.0000
Epoch 7/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0479 - accuracy: 1.0000
Epoch 8/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0316 - accuracy: 1.0000
Epoch 9/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0219 - accuracy: 1.0000
Epoch 10/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 11/45
42/42 [

/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

Epoch 1/45
42/42 [==============================] - 1s 3ms/step - loss: 0.5772 - accuracy: 1.0000
Epoch 2/45
42/42 [==============================] - 0s 3ms/step - loss: 0.4529 - accuracy: 1.0000
Epoch 3/45
42/42 [==============================] - 0s 3ms/step - loss: 0.3223 - accuracy: 1.0000
Epoch 4/45
42/42 [==============================] - 0s 3ms/step - loss: 0.2108 - accuracy: 1.0000
Epoch 5/45
42/42 [==============================] - 0s 3ms/step - loss: 0.1312 - accuracy: 1.0000
Epoch 6/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0811 - accuracy: 1.0000
Epoch 7/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0515 - accuracy: 1.0000
Epoch 8/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0343 - accuracy: 1.0000
Epoch 9/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0239 - accuracy: 1.0000
Epoch 10/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0175 - accuracy: 1.0000
Epoch 11/45
42/42 [

/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

Epoch 1/45
42/42 [==============================] - 1s 4ms/step - loss: 0.5750 - accuracy: 1.0000
Epoch 2/45
42/42 [==============================] - 0s 3ms/step - loss: 0.4492 - accuracy: 1.0000
Epoch 3/45
42/42 [==============================] - 0s 3ms/step - loss: 0.3158 - accuracy: 1.0000
Epoch 4/45
42/42 [==============================] - 0s 3ms/step - loss: 0.2030 - accuracy: 1.0000
Epoch 5/45
42/42 [==============================] - 0s 3ms/step - loss: 0.1245 - accuracy: 1.0000
Epoch 6/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0763 - accuracy: 1.0000
Epoch 7/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0480 - accuracy: 1.0000
Epoch 8/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 9/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0220 - accuracy: 1.0000
Epoch 10/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0160 - accuracy: 1.0000
Epoch 11/45
42/42 [

/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.8/dist-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

Epoch 1/45
42/42 [==============================] - 1s 3ms/step - loss: 0.5759 - accuracy: 1.0000
Epoch 2/45
42/42 [==============================] - 0s 3ms/step - loss: 0.4498 - accuracy: 1.0000
Epoch 3/45
42/42 [==============================] - 0s 3ms/step - loss: 0.3167 - accuracy: 1.0000
Epoch 4/45
42/42 [==============================] - 0s 3ms/step - loss: 0.2033 - accuracy: 1.0000
Epoch 5/45
42/42 [==============================] - 0s 3ms/step - loss: 0.1241 - accuracy: 1.0000
Epoch 6/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0758 - accuracy: 1.0000
Epoch 7/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0477 - accuracy: 1.0000
Epoch 8/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0314 - accuracy: 1.0000
Epoch 9/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0218 - accuracy: 1.0000
Epoch 10/45
42/42 [==============================] - 0s 3ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 11/45
42/42 [

In [ ]:
results

{'fit_time': array([27.72906375, 23.87133932, 23.33851194, 26.57362437]),
 'score_time': array([0.21316743, 0.17258048, 0.17226672, 0.167027  ]),
 'test_score': array([0.95495495, 0.95495495, 0.97297297, 0.97747748]),
 'train_score': array([1., 1., 1., 1.])}

In [ ]:
np.mean(results['test_score'])

0.9650900900900901